In [1]:
!pip install transformers datasets scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('/kaggle/input/train-dataset2-csv/train_dataset2.csv')
df

,text,status
0,oh my gosh,Anxiety
1,trouble sleeping confused mind restless heart ...,Anxiety
2,all wrong back off dear forward doubt stay in ...,Anxiety
3,i have shifted my focus to something else but ...,Anxiety
4,i am restless and restless it is been a month ...,Anxiety
...,...,...
89995,tw strong arm abuse my dad was screaming at me...,Stress
89996,hi i cannot think clearly today i know i have ...,Stress
89997,my chest give a dissimilar aroma before it wou...,Stress
89998,he was going to choke the name out of me i am ...,Stress


In [5]:
from sklearn.preprocessing import LabelEncoder

# Load and encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['status'])

# Get label names and mapping
label_names = label_encoder.classes_
num_labels = len(label_names)

# Optional: Check label mappings
label_mapping = dict(zip(label_names, range(num_labels)))
print(label_mapping)

{'Anxiety': 0, 'BPD': 1, 'Normal': 2, 'Stress': 3, 'Suicidal': 4, 'bipolar': 5, 'depression': 6, 'mentalillness': 7, 'schizophrenia': 8}


In [6]:
from datasets import Dataset

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df[['text', 'label']])
dataset = dataset.train_test_split(test_size=0.2)  # Split into train/test

In [ ]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"  # Use bart-base for faster fine-tuning than bart-large-mnli
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments, Trainer
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.006,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_35/1637027611.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.287200,1.174998,0.609278
1000,1.265800,1.091090,0.634889
1500,0.929900,1.039384,0.649389
2000,1.214800,1.001145,0.662000
2500,1.046100,0.960062,0.675056
3000,0.892900,0.946662,0.679611
3500,1.037800,0.942225,0.679389
4000,1.017700,0.925471,0.678722
4500,0.919200,0.909515,0.688000
5000,0.789400,0.928519,0.689833


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=13500, training_loss=0.7952181755348489, metrics={'train_runtime': 3755.7947, 'train_samples_per_second': 57.511, 'train_steps_per_second': 3.594, 'total_flos': 7154132502528000.0, 'train_loss': 0.7952181755348489, 'epoch': 3.0})

In [14]:
from sklearn.metrics import classification_report

predictions = trainer.predict(tokenized_datasets["test"])  # or ["validation"]
preds = predictions.predictions.argmax(axis=-1)
labels = predictions.label_ids

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [15]:
print(classification_report(labels, preds))

              precision    recall  f1-score   support

           0       0.75      0.77      0.76      2102
           1       0.67      0.69      0.68      1999
           2       0.88      0.89      0.88      1995
           3       0.90      0.89      0.89      1998
           4       0.66      0.65      0.66      2003
           5       0.69      0.70      0.69      1980
           6       0.57      0.53      0.55      2032
           7       0.47      0.48      0.47      1937
           8       0.71      0.71      0.71      1954

    accuracy                           0.70     18000
   macro avg       0.70      0.70      0.70     18000
weighted avg       0.70      0.70      0.70     18000



In [16]:
model.save_pretrained("./my_model")
tokenizer.save_pretrained("./my_model")

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json',
 './my_model/tokenizer.json')

In [17]:
!zip -r my_model.zip my_model

  adding: my_model/ (stored 0%)
  adding: my_model/config.json (deflated 55%)
  adding: my_model/tokenizer_config.json (deflated 75%)
  adding: my_model/vocab.txt (deflated 53%)
  adding: my_model/special_tokens_map.json (deflated 42%)
  adding: my_model/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: my_model/tokenizer.json (deflated 71%)


In [18]:
import shutil

shutil.make_archive('my_model', 'zip', '/kaggle/working/my_model')

'/kaggle/working/my_model.zip'

In [19]:
from IPython.display import FileLink
FileLink(r'my_model.zip')

/kaggle/working/my_model.zip